# TARA Ontology: SPARQL Query Examples
This notebook is created to demonstrate the utility of TARA ontologies through a set of SPARQL queries. To use this notebook you will need to install [jupyter notebook](https://jupyter.org/install) with [SPARQL kernel](https://github.com/paulovn/sparql-kernel).
* It is recommended that you have the basic understanding of the TARA Acupoints Ontology based on the documentation [linked here](https://github.com/smtifahim/TARA-Ontology-Repository/blob/master/ontology-files/generated/readme.md).

## Set Up the Stardog SPARQL Endpoint for TARA Ontology
To run the example queries, please set up the Stardog SPARQL Endpoint first, by running the following magics.

In [46]:
%endpoint https://sd-c1e74c63.stardog.cloud:5820/TARA-Acupoints/query
%auth basic TARA qy8CLEJXrwVqKNsC
%format any
%display table
%show 16

Endpoint set to: https://sd-c1e74c63.stardog.cloud:5820/TARA-Acupoints/query
HTTP authentication: method=basic, user=TARA, passwd set
Request format: ANY
Display: table
Result maximum size: 16

### Q1: List all acupoints along with their labels, synonyms, and Chiense names

In [38]:
# To save the query result in JSON format uncomment the following two lines. Specify the location and name of the JSON output after %outfile
# %format JSON
# %outfile sparql-results/Q1-Results.json

# List all acupoints with their label, synonyms, and chinese names.

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT ?acupoint_curie ?acupoint_label ?synonym ?chinese_name
WHERE 
{
    ## Partial match for an acupoint either directly or synonymously. Uncomment the following two lines.
    #  VALUES ?search_term { "fengmen" }
    #  FILTER ( REGEX(?acupoint_label, ?search_term, "i") || REGEX(?synonym, ?search_term, "i") || REGEX(?chinese_name, ?search_term, "i") )

    ?acupoint_iri rdfs:subClassOf+/rdfs:label 'Acupoint'.
    ?acupoint_iri rdfs:label ?acupoint_label.
    
    OPTIONAL { ?acupoint_iri TARA:hasSynonym ?synonym.}
    OPTIONAL { ?acupoint_iri TARA:hasChineseName ?chinese_name.}

    # Exclude generic categorizations like Acupoint of the X Meridian, Meridian Acupoint, Special Points, etc.
    FILTER (!regex(str(?acupoint_label), 'Acupoint of|Meridian Acupoint|Point of|Point|Extra Acupoint'))
    
    # To have the curie form of the acupoint_iri in the query result. 
    # Jupyter's SPARQL kernel does not return the curie form by default.
    BIND (REPLACE(STR(?acupoint_iri), "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
                                      "TARA:") AS ?acupoint_curie)
}
ORDER BY ?acupoint_label
LIMIT 999

acupoint_curie,acupoint_label,synonym,chinese_name
TARA:TARA_3480360,Anmian,,
TARA:TARA_1460220,BL 1,Bladder 1,Jingming
TARA:TARA_0242146,BL 10,Bladder 10,Tianzhu
TARA:TARA_1985889,BL 11,Influential Point of Bone,Dazhu
TARA:TARA_1985889,BL 11,Bladder 11,Dazhu
TARA:TARA_5468609,BL 12,Bladder 12,Fengmen
TARA:TARA_6050541,BL 13,Bladder 13,Feishu
TARA:TARA_6050541,BL 13,Back-Shu Point of the Lung,Feishu
TARA:TARA_9523348,BL 14,Bladder 14,Jueyinshu
TARA:TARA_9523348,BL 14,Back-Shu Point of the Pericardium,Jueyinshu


### Q2: List all meridans with their labels and synonyms

In [32]:
# List all meridans with their labels and synonyms.

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT ?meridian_curie ?meridian_label ?synonym
WHERE 
{
    ## Partial match of the meridan either directly or synonymously;
    # VALUES ?search_term { "Du" }
    # FILTER (REGEX(?meridian_label, ?search_term, "i") || REGEX(?synonym, ?search_term, "i"))


    ?meridian_iri rdfs:subClassOf+/rdfs:label "Meridian".
    ?meridian_iri rdfs:label ?meridian_label.
    
    OPTIONAL { ?meridian_iri TARA:hasSynonym ?synonym. }
    OPTIONAL { ?meridian_iri TARA:hasDesignatedOrgan ?associated_organ. }

    # Exclude generic categorization classes
    FILTER (!regex(str(?meridian_label), 'Main Meridian|Extra Meridian'))

    BIND ( REPLACE(STR(?meridian_iri), 
           "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
           "TARA:") AS ?meridian_curie)
}
ORDER BY ?meridina_label
limit 999

meridian_curie,meridian_label,synonym
TARA:TARA_6789717,Conception Vessel,CV
TARA:TARA_6789717,Conception Vessel,RN
TARA:TARA_6789717,Conception Vessel,Ren Channel
TARA:TARA_7525476,Bladder Meridian,BL
TARA:TARA_7525476,Bladder Meridian,Bladder Meridian of Foot-Taiyang
TARA:TARA_7525476,Bladder Meridian,meridian of the bladder
TARA:TARA_6221269,Stomach Meridian,ST
TARA:TARA_6221269,Stomach Meridian,Stomach Meridian of Foot-Yangming
TARA:TARA_6221269,Stomach Meridian,meridan of the stomach
TARA:TARA_5283975,Gallbladder Meridian,GB


### Q3: List all the acupoints with meridans, locational description, acupuncture method, and reference.

In [43]:
# List all the acupoints with meridans, locational description, acupuncture method, and reference.

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT 
?acupoint_curie ?acupoint_label ?meridian 
?location_description ?acupuncture_method ?reference
WHERE 
{
    ## Partial match for an acupoint search
    #  VALUES ?search_term { "LI" }
    #  FILTER ( REGEX(?acupoint_label, ?search_term, "i") || REGEX(?meridian, ?search_term, "i") )

    ?acupoint_iri rdfs:subClassOf+/rdfs:label "Acupoint".
    ?acupoint_iri rdfs:label ?acupoint_label.
    
    OPTIONAL { ?acupoint_iri TARA:hasMeridian/rdfs:label ?meridian. }
    OPTIONAL { ?acupoint_iri TARA:hasLocationalDescription ?location_description. }
    OPTIONAL { ?acupoint_iri TARA:hasMethodDescription ?acupuncture_method. }
    OPTIONAL { ?acupoint_iri TARA:hasReference ?reference. }

    # Exclude generic categorization classes like 'Acupoint of the X Meridian'. Meridian Acupoint, 'Special Points', etc.
    FILTER (!regex(str(?acupoint_label), 'Acupoint of|Meridian Acupoint|Point of|Point|Extra Acupoint'))

    BIND (REPLACE(STR(?acupoint_iri), 
          "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
          "TARA:") AS ?acupoint_curie)
}
ORDER BY ?acupoint_label
limit 999

acupoint_curie,acupoint_label,meridian,location_description,acupuncture_method,reference
TARA:TARA_3480360,Anmian,,Midpoint between Yifeng (TE 17) and Fengchi (GB 20).,Puncture perpendicularly 0.5-0.8 inch.,"Chapter 10 of ""Chinese Acupuncture and Moxibustion” (1999) ISBN-13: 978-7119017599"
TARA:TARA_1460220,BL 1,Bladder Meridian,"On the face, in the depression between the superomedial parts of the inner canthus of the eye and the medial wall of the orbit. Note: When the eye is closed, BL1 is located in the depression 0.1 B-cun superior and medial to the inner canthus of the eye.","Ask the patient to close his eyes when pushing gently the eyeball to the lateral side. Puncture slowly perpendicularly 0.3-0.7 inch along the orbital wall. It is not advisable to twist of lift and thrust the needle vigorously. To avoid bleeding, press the puncturing site for a few seconds after withdrawal of the needle. Moxibustion is forbidden.","WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 100"
TARA:TARA_0242146,BL 10,Bladder Meridian,"In the posterior region of the neck, at the same level as the superior border of the spinous process of the second cervical vertebra (C2), in the depression lateral to the trapezius muscle.",Puncture perpendicularly 0.5-0.8 inch.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 104"
TARA:TARA_1985889,BL 11,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous process of the first thoracic vertebra (T1), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 105"
TARA:TARA_5468609,BL 12,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous process of the second thoracic vertebra (T2), 1.5 B- cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 105"
TARA:TARA_6050541,BL 13,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous pro- cess of the third thoracic vertebra (T3), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 05-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 106"
TARA:TARA_9523348,BL 14,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous process of the fourth thoracic vertebra (T4), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 106"
TARA:TARA_5797983,BL 15,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous pro- cess of the fifth thoracic vertebra (T5), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 107"
TARA:TARA_1104010,BL 16,Bladder Meridian,"In the upper back region, level with the inferior border of the spinous process of the sixth thoracic vertebra (T6), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture 

### Q4: List all the acupoints along with their meridians and special point roles 

In [47]:
# List all the acupoints along with their meridians and special point roles

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT
?acupoint_curie ?acupoint_label ?meridian
?special_point_role ?special_point_category

WHERE 
{
    # FILTER (?acupoint = 'LU 9').
    # FILTER (?meridian = 'Lung Meridian').
    # FILTER (?special_point_category = 'Front-Mu Point').
    # FILTER (?special_point_category = 'Xi-Cleft Point').

    OPTIONAL {?acupoint_iri TARA:hasMeridian/rdfs:label ?meridian.}
    ?acupoint_iri rdfs:subClassOf*/rdfs:label "Meridian Acupoint".
    ?acupoint_iri rdfs:label ?acupoint_label.

    OPTIONAL
    {
        ?acupoint_iri rdfs:subClassOf+ ?special_point_iri.        
        ?special_point_iri rdfs:subClassOf+ ?special_point_category_iri.
        ?special_point_category_iri rdfs:subClassOf+/rdfs:label 'Special Point'.
         
        ?special_point_iri rdfs:label ?special_point_role. 
        ?special_point_category_iri rdfs:label ?special_point_category.
    }

   # Exclude generic categorization classes like 'Acupoint of the X Meridian'. Meridian Acupoint, 'Special Points', etc.
    FILTER (!regex(str(?acupoint_label), 'Acupoint of|Meridian Acupoint|Point of|Point|Extra Acupoint'))
    
    # To have the curie form of the acupoint_iri in the query result. 
    # Jupyter's SPARQL kernel does not return the curie form by default.
    BIND (REPLACE(STR(?acupoint_iri), "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
                                      "TARA:") AS ?acupoint_curie)
}
ORDER BY ?meridian ?acupoint
limit 999

acupoint_curie,acupoint_label,meridian,special_point_role,special_point_category
TARA:TARA_4839095,BL 21,Bladder Meridian,Back-Shu Point of the Stomach,Special Point on the Head and Trunk
TARA:TARA_4839095,BL 21,Bladder Meridian,Back-Shu Point of the Stomach,Back-Shu Point
TARA:TARA_4839095,BL 21,Bladder Meridian,Back-Shu Point,Special Point on the Head and Trunk
TARA:TARA_4995021,BL 59,Bladder Meridian,Xi-Cleft Point,Special Point on the Limbs
TARA:TARA_4995021,BL 59,Bladder Meridian,Xi-Cleft Point of the Yang Heel Vessel,Special Point on the Limbs
TARA:TARA_4995021,BL 59,Bladder Meridian,Xi-Cleft Point of the Yang Heel Vessel,Xi-Cleft Point
TARA:TARA_3963382,BL 22,Bladder Meridian,Back-Shu Point of Triple Energizer,Special Point on the Head and Trunk
TARA:TARA_4384020,BL 26,Bladder Meridian,,
TARA:TARA_3963382,BL 22,Bladder Meridian,Back-Shu Point,Special Point on the Head and Trunk
TARA:TARA_4648540,BL 55,Bladder Meridian,,


### Q5: List all acupoints along with their meridians, surface locations, and body regions.

In [49]:
# List all the acupoints along with their meridians, surface locations, and body regions

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT 
?acupoint_curie ?acupoint_label ?meridian
?surface_region
?related_region
?body_region

WHERE 
{
   # FILTER (REGEX (?acupoint_label, 'LU')).
   # FILTER (?meridian = "Lung Meridian")
   # FILTER (?body_region = 'chest').
   # FILTER (?body_region = 'leg').
   # FILTER (?surface_region = 'pedal digit 1').

    OPTIONAL {?acupoint_iri TARA:hasMeridian/rdfs:label ?meridian.}
    ?acupoint_iri rdfs:subClassOf*/rdfs:label "Meridian Acupoint".
    ?acupoint_iri rdfs:label ?acupoint_label.

    OPTIONAL 
    {   
        ?acupoint_iri TARA:hasSurfaceLocation ?surface_region_iri.
        ?surface_region_iri rdfs:label ?surface_region.
    }

    OPTIONAL
    {
        ?acupoint_iri TARA:hasRelatedLocation ?related_region_iri.
        ?related_region_iri rdfs:label ?related_region.
    }

    OPTIONAL
    {
    ?surface_region_iri ilxtr:isPartOf*/rdfs:label ?body_region.
    FILTER (?body_region in ('head', 'neck', 'back', 'chest', 'abdomen',
                             'limb', 'perineum', 'lower limb segment', 'upper limb segment', 
                             'hand', 'arm', 'face', 'leg', 'foot', 'knee')).
    }

    FILTER (?surface_region != ?related_region)

    # Exclude generic categorization classes like 'Acupoint of the X Meridian'. Meridian Acupoint, 'Special Points', etc.
    FILTER (!regex(str(?acupoint_label), 'Acupoint of|Meridian Acupoint|Point of|Point|Extra Acupoint'))
   
    # To have the curie form of the acupoint_iri in the query result. 
    # Jupyter's SPARQL kernel does not return the curie form by default.
    BIND (REPLACE(STR(?acupoint_iri), "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
                                      "TARA:") AS ?acupoint_curie) 
}
ORDER BY ?acupoint_label
limit 999

acupoint_curie,acupoint_label,meridian,surface_region,related_region,body_region
TARA:TARA_1460220,BL 1,Bladder Meridian,face,inner canthus,head
TARA:TARA_1460220,BL 1,Bladder Meridian,face,inner canthus,face
TARA:TARA_1460220,BL 1,Bladder Meridian,face,orbital cavity,head
TARA:TARA_1460220,BL 1,Bladder Meridian,face,orbital cavity,face
TARA:TARA_1460220,BL 1,Bladder Meridian,face,camera-type eye,head
TARA:TARA_1460220,BL 1,Bladder Meridian,face,camera-type eye,face
TARA:TARA_0242146,BL 10,Bladder Meridian,neck,trapezius muscle,neck
TARA:TARA_0242146,BL 10,Bladder Meridian,neck,vertebral bone 2,neck
TARA:TARA_1985889,BL 11,Bladder Meridian,dorsal thoracic segment of trunk,thoracic vertebra 1,
TARA:TARA_5468609,BL 12,Bladder Meridian,dorsal thoracic segment of trunk,thoracic vertebra 2,
